In [ ]:
import os
import requests

from bs4 import BeautifulSoup
from dotenv import load_dotenv
from slack_sdk import WebClient
from openai import OpenAI

load_dotenv()

### OpenAI API 사용해보기

In [ ]:
def get_chatgpt_message(client, message):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": message},
        ],
        temperature=0.3,
    )
    
    output = response.choices[0].message.content
    return output

### Slack Message 함수

In [ ]:
def send_slack_message(client, channel_id, message):
    news_title = ""
    for msg in message:
        news_title += f"{msg}\n"
    
    message_form = f"""
    어제의 뉴스 chatGPT 관련 뉴스 입니다.
    {news_title}
    """
    client.chat_postMessage(channel=channel_id, text=message)
    

### 네이버 뉴스 ChatGPT 관련 뉴스

In [ ]:
url = "https://search.naver.com/search.naver?where=news&query=ChatGPT&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2024.07.17&de=2024.07.18&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20240717to20240718&is_sug_officeid=0&office_category=0&service_area=0"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

contents_list = soup.select('.news_contents')

news_contents = []

for content in contents_list:
    title = content.select_one('.news_tit')
    dsc = content.select_one('.news_dsc')
    href = content.a['href']
    
    news_dict = {
        "title": title.text,
        "dsc": dsc.text,
        "href": href
    }
    
    news_contents.append(news_dict)

In [ ]:
news_text = ""
for content in news_contents:
    news_text += f"제목: {content['title']}\n"
    news_text += f"링크: {content['href']}\n"
    news_text += f"내용: {content['dsc']}\n"

message_form = f"""
어제의 뉴스 chatGPT 관련 뉴스 입니다.
------------Top 10-------------
{news_text}
"""
print(message_form)

### ChatGPT 프롬프트 작성

In [ ]:
openai_message = f"""
아래 <뉴스 리스트>의 내용을 3줄로 요약해줘.

<뉴스 리스트>
{news_text}
</뉴스 리스트>
"""

print(openai_message.strip())

### GPT-3.5-turbo에 프롬프팅하기

In [ ]:
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

gpt_summary = get_chatgpt_message(client=openai_client, message=openai_message)

In [ ]:
print(gpt_summary)

### Slack bot으로 보내기

In [ ]:
slack_client = WebClient(token=os.getenv("SLACK_TOKEN"))
channel_id="C077GUCPK0B"

slack_message = f"""
어제의 ChatGPT 관련 뉴스 3줄 요약 입니다!
{gpt_summary}
"""
send_slack_message(client=slack_client, channel_id=channel_id, message=slack_message)